# Example 1: Symmetrization on beam-splitter
In this example we analyze what happens if two polarized indistinguishable 
photons mix at the general beam splitter with reflectances $T_h$ and $T_v$ for
each polarization.

We will see that the ballanced beam splitter would project the state
to the singlet Bell state.

In [1]:
import json
import numpy as np
import sympy as sp
from json_netlist import instantiate_netlist_components, sp_calculate_effective_matrix
from two_photon_conversion import sp_construct_operator, sp_create_filter_matrix

mode numbering:
* 0 : path A, polarization H
* 1 : path A, polarization V
* 2 : path B, polarization H
* 3 : path B, polarization V
it is same for input and output

In [2]:
netlist_json_string = """
{
    "CentralBS" : {
      "type" : "BS2pol",
      "input_modes" : [0,1,2,3],
      "output_modes" : [0,1,2,3],
      "layer" : 0,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    }
  }
"""
netlist = json.loads(netlist_json_string)

instances = instantiate_netlist_components(netlist)
single_photon_map = sp_calculate_effective_matrix(instances)
single_photon_map #print single photon map

Matrix([
[     sqrt(Th),             0, sqrt(1 - Th),            0],
[            0,      sqrt(Tv),            0, sqrt(1 - Tv)],
[-sqrt(1 - Th),             0,     sqrt(Th),            0],
[            0, -sqrt(1 - Tv),            0,     sqrt(Tv)]])

how computational basis state are encoded:
* |00> : signal photon in mode 0 (path A, polarization H), idler photon in mode 2 (path B, polarization H)
* |01> : signal photon in mode 0 (path A, polarization H), idler photon in mode 3 (path B, polarization V)
* |10> : signal photon in mode 1 (path A, polarization V), idler photon in mode 2 (path B, polarization H)
* |11> : signal photon in mode 1 (path A, polarization V), idler photon in mode 3 (path B, polarization V)
the output encoding is the same.

In [3]:
allowed_inputs = [
    (0,2),
    (0,3),
    (1,2),
    (1,3)
]
allowed_outputs = [
    (0,2),
    (0,3),
    (1,2),
    (1,3)
]
n_map_rows = single_photon_map.shape[0]
filter_matrix = sp_create_filter_matrix(allowed_outputs, n_map_rows, True, True)
two_photon_operator = sp_construct_operator(single_photon_map, allowed_inputs, filter_matrix)
two_photon_operator #print single photon map

Matrix([
[2.0*Th - 1.0,                              0,                              0,            0],
[           0,          1.0*sqrt(Th)*sqrt(Tv), -1.0*sqrt(1 - Th)*sqrt(1 - Tv),            0],
[           0, -1.0*sqrt(1 - Th)*sqrt(1 - Tv),          1.0*sqrt(Th)*sqrt(Tv),            0],
[           0,                              0,                              0, 2.0*Tv - 1.0]])

In [4]:
#print substituted expression to see that it performs projection
#to singlet Bell state
sp.N(two_photon_operator.subs([('Th', 1/2), ('Tv', 1/2)] ),chop=1e-12)

Matrix([
[0,    0,    0, 0],
[0,  0.5, -0.5, 0],
[0, -0.5,  0.5, 0],
[0,    0,    0, 0]])

In [5]:
#plot twist: controlled pi-phase shift
sp.N(two_photon_operator.subs([('Th', 1), ('Tv', 1/3)] ),chop=1e-12)

Matrix([
[1.0,                 0,                 0,                  0],
[  0, 0.577350269189626,                 0,                  0],
[  0,                 0, 0.577350269189626,                  0],
[  0,                 0,                 0, -0.333333333333333]])